# Modelo Usuario - Usuario con mejores parametros

In [51]:
import numpy as np
import pandas as pd
import time
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import pickle
import warnings
warnings.filterwarnings("ignore")

# Carga Matriz Ratings

In [29]:
%cd D:/Dataset_1/lastfm-dataset-1K/\Modelo_Usuario_Usuario
#Carga Dataframe Ratings
df_ratings = pd.read_pickle("Ratings_Normalizacion_Log.pkl")
df_ratings.rating_lineal = round(df_ratings.rating_lineal)
df_ratings['rating_lineal'] = df_ratings['rating_lineal'].astype('uint8')
df_ratings.head(5)

D:\Dataset_1\lastfm-dataset-1K\Modelo_Usuario_Usuario


,user_id,artname,rating_lineal
0,0,0,1
1,1,1,1
2,2,2,1
3,3,3,1
4,3,4,1


# Carga de resultados de evaluación previamente obtenidos

In [7]:
df_resultados_usuario = pd.read_pickle("Evaluacion_Cos_Pearson_ModeloUser.pkl")
#Obtención de mejores parametros para el modelo
df_best_params = df_resultados_usuario.sort_values('RMSE', ascending = True).head(1)
best_k = df_best_params['Tamanio_Vecindario'].values[0]
Umbral = df_best_params['Umbral_Similitud'].values[0]
Medida = df_best_params['Indice'].values[0]

# Aplicación del modelo

In [30]:
reader = Reader( rating_scale = ( 1, 5 ) )
dataset_ratings = Dataset.load_from_df( df_ratings[ [ 'user_id', 'artname', 'rating_lineal']], reader )
# Particionamiento de Dataframe 80% entrenamiento y 20% Validación
train_set, test_set=  train_test_split(dataset_ratings, test_size=.2)
# Configuración de parametros
sim_options = {'name': Medida,'user_based': True , 'min_support': Umbral}
model = KNNBasic(k=best_k, min_k=2, sim_options=sim_options)

#Se le pasa la matriz de utilidad al algoritmo, es decir, el conjunto de entrenamiento
model.fit(trainset=train_set)

# Cálculo de predicción para todos los items del conjunto de validación
test_predictions=model.test(test_set)

#df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.uid, x.iid, x.est) , test_predictions)), columns=labels)
accuracy.rmse( test_predictions, verbose = True )

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.6260


0.6259839906510922

# Exportación Modelo

In [62]:
filename = 'Model_User_User.pkl'
pickle.dump(model, open(filename, 'wb'))

# Prueba Importación y predicción con parametros

In [68]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(8, 2843)
print(result)

user: 8          item: 2843       r_ui = None   est = 1.42   {'actual_k': 24, 'was_impossible': False}
